##### Copyright 2019 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Pix2Pix

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/generative/pix2pix"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/generative/pix2pix.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/generative/pix2pix.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/generative/pix2pix.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

Это руководство демонстрирует преобразование изображение в изображение с использованием условных GAN, как описано в [Преобразование изображения в изображение с использованием условных состязательных сетей](https://arxiv.org/abs/1611.07004). Используя эту технику, мы можем раскрасить черно-белые фотографии, преобразовать карты Google в Google Earth и т. д. В этом руководстве мы преобразовываем фасады зданий в настоящие здания.

В примере мы будем использовать [База данных фасадов CMP](http://cmp.felk.cvut.cz/~tylecr1/facade/), любезно предоставленная [Центром машинного восприятия](http://cmp.felk .cvut.cz/) в [Чешском техническом университете в Праге](https://www.cvut.cz/). Чтобы наш пример был кратким, мы будем использовать предварительно обработанную [копию](https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/) этого набора данных, созданную авторами [paper](https://arxiv.org/abs/1611.07004) выше.

Каждая эпоха занимает около 15 секунд на одном графическом процессоре V100.

Ниже приведен результат, полученный после обучения модели для 200 эпох.

![пример вывода_1](https://www.tensorflow.org/images/gan/pix2pix_1.png)
![пример вывода_2](https://www.tensorflow.org/images/gan/pix2pix_2.png)

## Импорт TensorFlow и других библиотек

In [ ]:
import tensorflow as tf

import os
import time

from matplotlib import pyplot as plt
from IPython import display

In [ ]:
!pip install -U tensorboard

## Загрузка датасета

Вы можете загрузить этот набор данных и аналогичные ему [здесь](https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets). Как упоминалось в [статье](https://arxiv.org/abs/1611.07004), мы применяем случайную рябь и зеркальное отображение к набору обучающих данных.

* При случайной ряби размер изображения изменяется до `286 x 286`, а затем случайным образом обрезается до `256 x 256`.
* При случайном зеркальном отображении изображение переворачивается по горизонтали, т.е. слева направо.

In [ ]:
_URL = 'https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/facades.tar.gz'

path_to_zip = tf.keras.utils.get_file('facades.tar.gz',
                                      origin=_URL,
                                      extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'facades/')

In [ ]:
BUFFER_SIZE = 400
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256

In [ ]:
def load(image_file):
  image = tf.io.read_file(image_file)
  image = tf.image.decode_jpeg(image)

  w = tf.shape(image)[1]

  w = w // 2
  real_image = image[:, :w, :]
  input_image = image[:, w:, :]

  input_image = tf.cast(input_image, tf.float32)
  real_image = tf.cast(real_image, tf.float32)

  return input_image, real_image

In [ ]:
inp, re = load(PATH+'train/100.jpg')
# преобразуем в int чтобы matplotlib мог показать изображение
plt.figure()
plt.imshow(inp/255.0)
plt.figure()
plt.imshow(re/255.0)

In [ ]:
def resize(input_image, real_image, height, width):
  input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  real_image = tf.image.resize(real_image, [height, width],
                               method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  return input_image, real_image

In [ ]:
def random_crop(input_image, real_image):
  stacked_image = tf.stack([input_image, real_image], axis=0)
  cropped_image = tf.image.random_crop(
      stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image[0], cropped_image[1]

In [ ]:
# нормализуем изображение до [-1, 1]

def normalize(input_image, real_image):
  input_image = (input_image / 127.5) - 1
  real_image = (real_image / 127.5) - 1

  return input_image, real_image

In [ ]:
@tf.function()
def random_jitter(input_image, real_image):
  # ресайз до 286 x 286 x 3
  input_image, real_image = resize(input_image, real_image, 286, 286)

  # случайная обреска до 256 x 256 x 3
  input_image, real_image = random_crop(input_image, real_image)

  if tf.random.uniform(()) > 0.5:
    # случайное зеркальное отображение
    input_image = tf.image.flip_left_right(input_image)
    real_image = tf.image.flip_left_right(real_image)

  return input_image, real_image

Как вы можете видеть на изображениях ниже,- они проходят через случайное зашумливание
Случайное зашумливание, описанное в документе, должно

1. Изменить размер изображения на больший.
2. Произвольно обрезать до нужного размера.
3. Произвольно перевернуть изображение по горизонтали.

In [ ]:
plt.figure(figsize=(6, 6))
for i in range(4):
  rj_inp, rj_re = random_jitter(inp, re)
  plt.subplot(2, 2, i+1)
  plt.imshow(rj_inp/255.0)
  plt.axis('off')
plt.show()

In [ ]:
def load_image_train(image_file):
  input_image, real_image = load(image_file)
  input_image, real_image = random_jitter(input_image, real_image)
  input_image, real_image = normalize(input_image, real_image)

  return input_image, real_image

In [ ]:
def load_image_test(image_file):
  input_image, real_image = load(image_file)
  input_image, real_image = resize(input_image, real_image,
                                   IMG_HEIGHT, IMG_WIDTH)
  input_image, real_image = normalize(input_image, real_image)

  return input_image, real_image

## Входной конвейер

In [ ]:
train_dataset = tf.data.Dataset.list_files(PATH+'train/*.jpg')
train_dataset = train_dataset.map(load_image_train,
                                  num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE)

In [ ]:
test_dataset = tf.data.Dataset.list_files(PATH+'test/*.jpg')
test_dataset = test_dataset.map(load_image_test)
test_dataset = test_dataset.batch(BATCH_SIZE)

## Создание генератора
   * Архитектура генератора - это модифицированная U-Net.
   * Каждый блок в энкодере(Conv -> Batchnorm -> Leaky ReLU)
   * Каждый блок в декодере(Transposed Conv -> Batchnorm -> Dropout (применяется к первым трем блокам) -> ReLU)
   * Между энкодером и декодером есть пропускаемые соединения(как в U-Net).


In [ ]:
OUTPUT_CHANNELS = 3

In [ ]:
def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [ ]:
down_model = downsample(3, 4)
down_result = down_model(tf.expand_dims(inp, 0))
print (down_result.shape)

In [ ]:
def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

In [ ]:
up_model = upsample(3, 4)
up_result = up_model(down_result)
print (up_result.shape)

In [ ]:
def Generator():
  inputs = tf.keras.layers.Input(shape=[256,256,3])

  down_stack = [
    downsample(64, 4, apply_batchnorm=False), # (bs, 128, 128, 64)
    downsample(128, 4), # (bs, 64, 64, 128)
    downsample(256, 4), # (bs, 32, 32, 256)
    downsample(512, 4), # (bs, 16, 16, 512)
    downsample(512, 4), # (bs, 8, 8, 512)
    downsample(512, 4), # (bs, 4, 4, 512)
    downsample(512, 4), # (bs, 2, 2, 512)
    downsample(512, 4), # (bs, 1, 1, 512)
  ]

  up_stack = [
    upsample(512, 4, apply_dropout=True), # (bs, 2, 2, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 4, 4, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 8, 8, 1024)
    upsample(512, 4), # (bs, 16, 16, 1024)
    upsample(256, 4), # (bs, 32, 32, 512)
    upsample(128, 4), # (bs, 64, 64, 256)
    upsample(64, 4), # (bs, 128, 128, 128)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh') # (bs, 256, 256, 3)

  x = inputs

  # понижение размерности
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Повышение размерности и установление пропуска соединений
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
generator = Generator()
tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)

In [ ]:
gen_output = generator(inp[tf.newaxis,...], training=False)
plt.imshow(gen_output[0,...])

* **Расчет потерь в генераторе**
   * Это сигмоидная кросс-энтропия сгенерированных изображений и **массива из них**.
   * [Статья](https://arxiv.org/abs/1611.07004) также включает потерю L1, которая представляет собой MAE(средняя абсолютная ошибка) между сгенерированным изображением и целевым изображением.
   * Это позволяет сгенерированному изображению стать структурно похожим на целевое изображение.
   * Формула для расчета общих потерь генератора loss = gan_loss + LAMBDA * l1_loss, где LAMBDA = 100. Это значение было определено авторами [статьи](https://arxiv.org/abs/1611.07004).

Процедура обучения генератора показана ниже:

In [ ]:
LAMBDA = 100

In [ ]:
def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

  # mean absolute error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

  total_gen_loss = gan_loss + (LAMBDA * l1_loss)

  return total_gen_loss, gan_loss, l1_loss

![Generator Update Image](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/gen.png?raw=1)


## Создание дискриминатора
   * Дискриминатор - это PatchGAN.
   * Каждый блок в дискриминаторе это (Conv -> BatchNorm -> Leaky ReLU)
   * Размерность вывода после последнего слоя: (batch_size, 30, 30, 1)
   * Каждый патч 30x30 на выходе классифицирует часть входного изображения размером 70x70 (такая архитектура называется PatchGAN).
   * Дискриминатор получает 2 входа.
     * Входное изображение и целевое изображение, которое следует классифицировать как реальное.
     * Входное изображение и сгенерированное изображение(вывод генератора), которое следует классифицировать как подделку.
     * Мы объединяем эти 2 ввода вместе в коде (`tf.concat ([inp, tar], axis = -1)`)

In [ ]:
def Discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[256, 256, 3], name='input_image')
  tar = tf.keras.layers.Input(shape=[256, 256, 3], name='target_image')

  x = tf.keras.layers.concatenate([inp, tar]) # (bs, 256, 256, channels*2)

  down1 = downsample(64, 4, False)(x) # (bs, 128, 128, 64)
  down2 = downsample(128, 4)(down1) # (bs, 64, 64, 128)
  down3 = downsample(256, 4)(down2) # (bs, 32, 32, 256)

  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3) # (bs, 34, 34, 256)
  conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1) # (bs, 31, 31, 512)

  batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 512)

  last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)

  return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [ ]:
discriminator = Discriminator()
tf.keras.utils.plot_model(discriminator, show_shapes=True, dpi=64)

In [ ]:
disc_out = discriminator([inp[tf.newaxis,...], gen_output], training=False)
plt.imshow(disc_out[0,...,-1], vmin=-20, vmax=20, cmap='RdBu_r')
plt.colorbar()

**Расчет потерь дискриминатора**
   * Функция потерь дискриминатора принимает 2 входа: **[реальные изображения, сгенерированные изображения]**
   * real_loss - это сигмоидная кросс-энтропия **реальных изображений** и **массива единиц(поскольку это настоящие изображения)**
   * generated_loss - сигмоидная кросс-энтропия **сгенерированных изображений** и **массива нулей(поскольку это поддельные изображения)**
   * В результате **total_loss** - это сумма real_loss и generated_loss

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

Процедура обучения дискриминатора показана ниже.

Чтобы узнать больше об архитектуре и гиперпараметрах, вы можете обратиться к [статье](https://arxiv.org/abs/1611.07004).

![Discriminator Update Image](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/dis.png?raw=1)


## Определение оптимайзера и сохранения чекпойнтов


In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Генерация изображения

Напишите функцию для построения изображений во время обучения.

* Мы передаем изображения из тестового датасета в генератор.
* Генератор преобразует входное изображение в выходное.
* Последний шаг - построить прогнозы и **вуаля!**

Примечание: аргумент `training` установлен в `True` намеренно, так как нам нужна пакетная статистика при запуске модели на тестовом наборе данных. 
Если мы используем `training = False`, мы получим накопленную статистику для всего набора данных(чего мы не хотим).

In [ ]:
def generate_images(model, test_input, tar):
  prediction = model(test_input, training=True)
  plt.figure(figsize=(15,15))

  display_list = [test_input[0], tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

In [ ]:
for example_input, example_target in test_dataset.take(1):
  generate_images(generator, example_input, example_target)

## Обучение

* Для каждого входного изображения генерируем выходное изображение.
* Дискриминатор получает входное и сгенерированное изображение в качестве первого входа. Второй вход - это входное и целевое изображения.
* Далее рассчитываем потери генератора и дискриминатора.
* Затем вычисляем градиенты потерь как для генератора, так и для переменных дискриминатора(входных данных) и применяем их к оптимизатору.
* Затем логируем потери в TensorBoard

In [ ]:
EPOCHS = 150

In [ ]:
import datetime
log_dir="logs/"

summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
@tf.function
def train_step(input_image, target, epoch):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(input_image, training=True)

    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True)

    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))

  with summary_writer.as_default():
    tf.summary.scalar('gen_total_loss', gen_total_loss, step=epoch)
    tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=epoch)
    tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=epoch)
    tf.summary.scalar('disc_loss', disc_loss, step=epoch)

Фактический цикл обучения:

* Итерирует по количеству эпох.
* В каждую эпоху очищает дисплей и запускает `generate_images`, чтобы показать прогресс.
* В каждую эпоху выполняет итерацию по набору тренировочных данных, печатая '.' для каждого примера.
* Сохраняет контрольную точку каждые 20 эпох.

In [ ]:
def fit(train_ds, epochs, test_ds):
  for epoch in range(epochs):
    start = time.time()

    display.clear_output(wait=True)

    for example_input, example_target in test_ds.take(1):
      generate_images(generator, example_input, example_target)
    print("Epoch: ", epoch)

    # Тренировка
    for n, (input_image, target) in train_ds.enumerate():
      print('.', end='')
      if (n+1) % 100 == 0:
        print()
      train_step(input_image, target, epoch)
    print()

    # сохраняем чекпойнт каждые 20 эпох
    if (epoch + 1) % 20 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                        time.time()-start))
  checkpoint.save(file_prefix = checkpoint_prefix)

Этот цикл обучения сохраняет логи, которые вы можете просматривать в TensorBoard для отслеживания прогресса обучения. Работая локально, вы запускаете отдельный процесс TensorBoard. В ноутбуке, если вы хотите контролировать процесс с помощью TensorBoard, проще всего запустить программу просмотра перед началом обучения.

Чтобы запустить программу просмотра, вставьте в ячейку кода следующее:

In [ ]:
#docs_infra: no_execute
%load_ext tensorboard
%tensorboard --logdir {log_dir}

Теперь запустите цикл обучения:

In [ ]:
fit(train_dataset, EPOCHS, test_dataset)

Если вы хотите опубликовать результаты TensorBoard _публично_, вы можете загрузить журналы в [TensorBoard.dev](https://tensorboard.dev/), скопировав и выполнив следующее в ячейку кода.

Примечание. Для этого требуется учетная запись Google.

```
!tensorboard dev upload --logdir  {log_dir}
```

Внимание! Эта команда не завершается. Он предназначен для постоянной загрузки результатов длительных экспериментов. После того, как ваши данные загружены, вам необходимо остановить выполнение команды, выполнив "interrupt execution" в вашем инструменте для работы с ноутбуком(jupyter, colab, etc).

Вы можете просмотреть [результаты предыдущего запуска](https://tensorboard.dev/experiment/lZ0C6FONROaUMfjYkVyJqw) этого ноутбука на [TensorBoard.dev](https://tensorboard.dev/).

TensorBoard.dev - это управляемый интерфейс для размещения, отслеживания и обмена экспериментами машинного обучения.

Он также может быть включен с помощью `<iframe>`:

In [ ]:
display.IFrame(
    src="https://tensorboard.dev/experiment/lZ0C6FONROaUMfjYkVyJqw",
    width="100%",
    height="1000px")

Интерпретация логов из GAN сложнее, чем интерпретация простой классификациии или регрессионной модели. На что следует обратить внимание:

* Убедитесь, что ни одна из моделей не «выиграла». Если либо `gen_gan_loss`, либо `disc_loss` становятся очень низким, это показатель того, что одна модель доминирует над другой, и вы не обучаете комбинированную модель успешно.
* Значение `log(2) = 0.69` является хорошим показателем для этих потерь, поскольку это значение указывает на неуверенность модели: дискриминатор в среднем одинаково неуверен в обоих вариантах.
* Для `disc_loss` значение ниже `0.69` означает, что дискриминатор работает лучше, чем случайно, на объединенном наборе реальных + сгенерированных изображений.
* Для `gen_gan_loss` значение ниже `0.69` означает, что генератор работает лучше, чем случайно, обманывая дескриминатор.
* По мере обучения значение `gen_l1_loss` должно уменьшаться.

## Восстановление последнего чекпойнта и тестирование

In [ ]:
!ls {checkpoint_dir}

In [ ]:
# восстанавливаем последний чекпойнт из checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## Генерация с использованием тестового датасета

In [ ]:
# Запускаем обученную модель на нескольких примерах из тестового набора данных
for inp, tar in test_dataset.take(5):
  generate_images(generator, inp, tar)